Write a python code in cocalc that implement the following scenario:

1) Read a stream of data from (data_stream.txt) file, one character at a time.

2) Generate the required number of bits using LFSR (generates 8-bit at a time for each character)

3) Perform the xor encryption.

4) In a separate function perform the xor decryption. Note: during decryption the sequence of key can be generated using the seed and mask value

 

5) OPTIONAL: this is an optional step. Try to crack the LFSR, by knowing the original data stream and the ciphertext.



LRSR generation function(from the lecture slides):

In [17]:
def LRSR_generate(seed, mask, n):
    seed_int = int(seed, 2)
    mask_int = int(mask, 2)
    nbits = len(seed)

    state = seed_int
    while n > 0:
        output = 1 & state
        _mask,_state,new_bit = mask_int, state, 0
        while _mask:
            new_bit ^= (1&_mask) * (1&_state)
            _mask >>= 1
            _state >>= 1
        state = state >> 1 | (new_bit << (nbits-1))

        yield output, state
        n-=1

Let's split the list of integers into 8 element chunks. The `list_split` function will do that for us.

In [18]:
def list_split(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [19]:
def convertListOfBinaryToChar(list):
    return chr(int("".join(map(str, list)),2))

To convert a list of integers into a string, we can `getKeyAsString` function. First we split the list into 8 element chunks, then we convert each chunk into a char and join the chars into a string. 

In [20]:
def getKeyAsString (key):
    keyLists = list(list_split ([ output for output, _ in key],8))
    return ''.join(convertListOfBinaryToChar(binaryList) for binaryList in keyLists)

Xor function(from earlier laboratories):

In [21]:
def xor(string1, string2):
    return "".join([chr(ord(c1) ^ ord(c2)) for (c1,c2) in zip(string1,string2)])

Read one character at a time from the file function. Reads using `read(1)` function(returns one character at a time)

In [22]:
def readFromFileOneCharAtATime(filename):
    with open(filename, "r") as f:
        while True:
            c = f.read(1)
            if not c:
                break
            yield c

Data stream file:

In [23]:
file = "data_stream.txt"

A small streamEncryption function, which takes filename, seed and mask as an input and returns xor encrypted data using LFSR function

In [40]:
def streamEncrypt(file,seed,mask):
    text = ''.join(str(output) for output in readFromFileOneCharAtATime(file))
    key = LRSR_generate(seed, mask, len(text) * 8)
    key_str = getKeyAsString(key)
    print (key_str)
    # print (key_str)
    print(key_str)
    encrypted_text = xor(text, key_str)
    return encrypted_text

In [41]:
# # seed = input('Enter seed: ')
# mask = input('Enter mask: ')
seed ='1001011001'
mask ='0101101101'

encrypted_text = streamEncrypt(file,seed,mask)
print(encrypted_text)

dC3Èr+ìvæPØ9¹`Ã½GÐ:Éoæ÷4Ègå$WØíÌ¡°2r5rÁz
dC3Èr+ìvæPØ9¹`Ã½GÐ:Éoæ÷4Ègå$WØíÌ¡°2r5rÁz
î*@èáV$ª|Xw¥#uk±%N¦Oî×õF©èöV%½ìÒÕQG­þT


A `streamDecprypt` function which takes encrypted text, seed and mask as an input and returns the decrypted text by using the LFSR function to generate the key and xor the encrypted text and key to retrieve the original text.

In [26]:
def streamDecrypt(encrypted_text, seed, mask):
    key = LRSR_generate(seed, mask, len(encrypted_text)*8)
    key_str = getKeyAsString(key)
    originalText = xor(encrypted_text, key_str)
    return originalText

In [27]:
print(streamDecrypt(encrypted_text, seed, mask))

this is a stream of data to be transferred securely.


Let's crack LRSR

In [28]:
def retrieveKey(encrypted_text, original_text):
    key = xor(encrypted_text, original_text)
    return key


original_text = ''.join(str(output)
                        for output in readFromFileOneCharAtATime(file))
retrievedKey = retrieveKey(encrypted_text, original_text)


Get binary string from the given key in char format

In [29]:
def getBinaryValues(key): ##('0100')
    keys=list(key)
    ones = [format(ord(k),'b').zfill(8) for k in keys]
    return ''.join(i for i in ones)

Let's try to get mask value from the key value. We can do this using berlekam massey algorithm:

In [30]:
import berlekampmassey

def GetMaskFromKey(key):
    binaryValues = getBinaryValues(key)
    ret = list(berlekampmassey.bm(binaryValues))[-1]
    return ''.join([str(x) for x in ret[::-1]])

Since the seed is inside the key, but don't know its length, we can iterate from 1 till the length of the key to find the seed value. Adter choosing `i` length seed from the key, we generate key length of values using LFSR and check if the generated key is equal to the given key. If it is equal, we found the seed value.

In [35]:
def GetSeedFromKey(key, mask):
    keyInBinary = getBinaryValues(key)
    for i in range(1, len(keyInBinary)):
        tmp_seed = keyInBinary[0:i][::-1]
        print(tmp_seed)
        tmp_key = LRSR_generate(tmp_seed, mask, len(keyInBinary)*16)
        tmp_key_str = ''.join(str(output) for output, _ in tmp_key)

        if (keyInBinary == tmp_key_str):
            return tmp_seed

Full seed, mask retrieval from the key:

In [36]:
solvedMask = GetMaskFromKey(retrievedKey)
print("mask comparison")
print("original mask")
print(mask)
print("retrieved mask from the key")
print(solvedMask)
print('\nseed comparison')
print("original seed")
print(seed)
print("retrieved seed from the key with solved mask")
print(GetSeedFromKey(retrievedKey, solvedMask))

mask comparison
original mask
0101101101
retrieved mask from the key
101101101

seed comparison
original seed
100101100
retrieved seed from the key with solved mask
0
00
100
1100
01100
101100
0101100
00101100
100101100
0100101100
10100101100
010100101100
1010100101100
01010100101100
101010100101100
1101010100101100
11101010100101100
011101010100101100
1011101010100101100
11011101010100101100
111011101010100101100
1111011101010100101100
11111011101010100101100
111111011101010100101100
1111111011101010100101100
01111111011101010100101100
001111111011101010100101100
1001111111011101010100101100
11001111111011101010100101100
011001111111011101010100101100
0011001111111011101010100101100
00011001111111011101010100101100
000011001111111011101010100101100
0000011001111111011101010100101100
00000011001111111011101010100101100
100000011001111111011101010100101100
0100000011001111111011101010100101100
10100000011001111111011101010100101100
010100000011001111111011101010100101100
...
None
